# Auxiliary Reuse and Management

This tutorial demonstrates automatic auxiliary qubits management by the synthesis engine. It synthesizes a simple state preparation function, which comprises several multi-controlled rotations. These rotations can use auxiliary qubits to reduce depth. For a given **global** width constraint, the initialization and reuse of auxiliary qubits between different function blocks is automated.

In [2]:
import numpy as np

from classiq import (
    GeneratedCircuit,
    QArray,
    QFunc,
    create_model,
    prepare_amplitudes,
    set_constraints,
    set_preferences,
    show,
    synthesize,
)
from classiq.model import (
    Constraints,
    CustomHardwareSettings,
    OptimizationParameter,
    Preferences,
)

NUM_QUBITS = 4
np.random.seed(12)
amplitudes = 1 - 2 * np.random.rand(2**NUM_QUBITS)
amplitudes = (amplitudes / np.linalg.norm(amplitudes)).tolist()


preferences = Preferences(
    custom_hardware_settings=CustomHardwareSettings(basis_gates=["cx", "u"]),
    random_seed=1235,
    optimization_timeout_seconds=100,
)


@QFunc
def main() -> None:
    out = QArray("out")
    prepare_amplitudes(amplitudes=amplitudes, bound=0.2, out=out)


qmod = create_model(main)
qmod = set_preferences(qmod, preferences)
constraints = Constraints(
    optimization_parameter=OptimizationParameter.DEPTH, max_width=8
)
qmod = set_constraints(qmod, constraints)
qprog = synthesize(qmod)

circuit = GeneratedCircuit.from_qprog(qprog)


show(qprog)

Opening: http://localhost:4200/circuit/a7612711-8d0a-4303-87c1-945cd3c10328?version=0.0.0
